In [11]:
import pandas as pd
import numpy as np

In [12]:
%run DataPreprocessing.ipynb

C:\Users\HP\AppData\Local\Temp\ipykernel_4844\3476190814.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_4844\3476190814.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_4844\3476190814.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)


In [13]:
latest_df = (
    final_df.sort_values("timestamp")
    .groupby("tunnel_id")
    .tail(1)
    .reset_index(drop=True)
)

In [14]:
connections = {
    "A": ["B"],
    "B": ["A", "C"],
    "C": ["B"]
}

In [15]:
# Map local risk label to numeric score
risk_map = {"Safe": 0.2, "Warning": 0.6, "Critical": 0.9}
latest_df["local_risk"] = latest_df["risk_label"].map(risk_map)

In [16]:
# Simple GNN-like risk propagation (message passing)
def propagate_risk(df, connections, influence=0.3):
    df = df.copy()
    df["gnn_risk"] = df["local_risk"]
    for i, row in df.iterrows():
        nbrs = connections.get(row["tunnel_id"], [])
        nbr_risk = df[df["tunnel_id"].isin(nbrs)]["local_risk"]
        if len(nbr_risk) > 0:
            df.loc[i, "gnn_risk"] += influence * nbr_risk.mean()
    return df

In [17]:
gnn_df = propagate_risk(latest_df, connections)

In [18]:
# Convert numeric risk to label
def label_from_score(x):
    if x > 0.8:
        return "Critical"
    elif x > 0.5:
        return "Warning"
    else:
        return "Safe"

gnn_df["gnn_risk_label"] = gnn_df["gnn_risk"].apply(label_from_score)


In [19]:
# Save GNN outputs for dashboard use
import joblib, os
os.makedirs("models", exist_ok=True)
joblib.dump(gnn_df[["tunnel_id", "gnn_risk", "gnn_risk_label"]], "models/gnn_output.pkl")

['models/gnn_output.pkl']

In [20]:
gnn_df[["tunnel_id", "local_risk", "gnn_risk", "gnn_risk_label"]]

,tunnel_id,local_risk,gnn_risk,gnn_risk_label
0,A,NaN,NaN,Safe
1,B,NaN,NaN,Safe
2,C,NaN,NaN,Safe
